## Atomistic Molecular Dynamics: Bulk Liquids - Linear Alkanes

The second system we will consider for performing atomistic MD is a system of linear alkanes.

### Build the system

Again, we will use mBuild to construct our system.

First, we will import mBuild, as well as specify a filter for some warnings that can often clutter the output.

In [ ]:
import mbuild as mb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Now, we will define a class to create a `CH2` moiety, similar to what we did on Monday. Here, we load the structure for a CH2 moeity from a PDB file and add two ports for the two dangling bonds on the carbon.

In [ ]:
class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        mb.load('utils/ch2.pdb', compound=self)
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.07), 'up')
        self.add(mb.Port(anchor=self[0], orientation=[0, -1, 0], separation=0.07), 'down')

Let's visualize to make sure it looks okay.

In [ ]:
ch2 = CH2()
ch2.visualize(show_ports=True)

Here, we create a hydrogen atom with a `Port` representing a single dangling bond.

In [ ]:
class Hydrogen(mb.Compound):
    def __init__(self):
        super(Hydrogen, self).__init__()
        self.add(mb.Compound(name='H'))
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.07), 'up')

Again, let's visualize as a sanity check.

In [ ]:
h = Hydrogen()
h.visualize(show_ports=True)

#### Create class for an alkane chain (with arbitrary length)

Here, we will define a class that stitches together CH2 moieties into an alkane chain of a user-defined length (hydrogen atoms are used to cap the ends of the chain to provide the complete chemistry). This should look familiar from our session on Monday.

We provide a `chain_length` argument to the constructor so that we can easily toggle the length of the chain upon instantiation.

In [ ]:
class Alkane(mb.Compound):
    """An alkane chain of a user-defined length."""
    def __init__(self, chain_length):
        """Initialize an Alkane Compound.

        Parameters
        ----------
        chain_length : int
            Length of the alkane chain (in number of carbons)
        """
        # Make sure the user inputs a chain length of at least 1
        if chain_length < 1:
            raise ValueError('Chain length must be greater than 1')
        
        super(Alkane, self).__init__()

        # Create a polymer of CH2 units
        chain = mb.Polymer(CH2(), n=chain_length, port_labels=('up', 'down'))
        self.add(chain, 'chain')
        
        # Cap one end of the polymer with a hydrogen
        self.add(Hydrogen(), 'up_cap')
        mb.force_overlap(move_this=self['up_cap'],
                         from_positions=self['up_cap']['up'],
                         to_positions=self['chain']['up'])
        
        # Cap the other end of the polymer with a hydrogen
        self.add(Hydrogen(), 'down_cap')
        mb.force_overlap(move_this=self['down_cap'],
                         from_positions=self['down_cap']['up'],
                         to_positions=self['chain']['down'])

Let's check out what happens if we pass a chain length of 1 to an instance of this class. We'll also perform an energy minimization so the resulting structure looks reasonable.

In [ ]:
methane = Alkane(chain_length=1) # Create an alkane with chain length of 1, i.e. methane
methane.energy_minimization()    # Energy minimize so that the structure appears reasonable
methane.visualize()

Again, as we've added the `chain_length` argument to the class definition. Creating a chain with a different length is as easy as creating another instance with a different `chain_length` value provided.

In [ ]:
hexane = Alkane(chain_length=6) # Create an alkane with chain length of 6, i.e. hexane
hexane.energy_minimization()    # Energy minimize so that the structure appears reasonable
hexane.visualize()

#### Create class for a box of alkane chains

Now we will define a class that creates a box of alkanes. This will be similar to the one we created on Monday; however, this time we will also allow for a `density` argument to be provided upon instantiation, in addition to arguments for the chain length and number of chains. This will tell mBuild to figure out the proper box size to be filled with the desired number of molecules at the desired density. This class then appears similar to the `WaterBox` class we defined in the other notebook.

In [ ]:
class AlkaneBox(mb.Compound):
    """An box of linear alkane chains."""
    def __init__(self, chain_length, n_chains, density):
        """Initialize an AlkaneBox Compound.

        Parameters
        ----------
        chain_length : int
            Length of the alkane chains (in number of carbons)
        n_chains : int
            Number of chains to place in the box
        density : float
            Density (in kg/m^3) at which the system should be created
        """
        super(AlkaneBox, self).__init__()
        
        # Create alkane chain prototype using the class above
        chain = Alkane(chain_length=chain_length)
        
        # Generate a more relaxed structure
        chain.energy_minimization()
        
        # Fill a box with chains at a user-defined density
        box_of_alkanes = mb.fill_box(compound=chain, n_compounds=n_chains,
                                     density=density)
        
        # Rename all chains to `Alkane`, this speeds up the atom-typing process
        for child in box_of_alkanes.children:
            child.name = 'Alkane'
        self.add(box_of_alkanes)

Let's go ahead and create an instance of the `AlkaneBox` class for a box of 250 hexane molecules. We will provide the experimentally expected density of 659 kg/m^3.

In [ ]:
hexane_box = AlkaneBox(chain_length=6, n_chains=250, density=659)

...and let's visualize to see how it looks.

In [ ]:
hexane_box.visualize()

Again we will need to write to both GRO and TOP formats. Regarding force field application, Foyer natively includes two force fields: the OPLS all-atom force field and the TraPPE united-atom force field. Since we have an all-atom system and the OPLS force field is designed for simulations of liquids, we will use that force field by providing the `forcefield_name='oplsaa'` argument when saving to the TOP file.

In [ ]:
hexane_box.save('hexane.gro', overwrite=True, residues='Alkane')
hexane_box.save('hexane.top', forcefield_name='oplsaa', overwrite=True, residues='Alkane')

Again, we will need to run `grompp` to check for errors and to create the binary TPR file. The MDP file we are using is identical to the one used for the water box, except for one additional line:

In [ ]:
!sed -n '7 p' utils/npt-alkane.mdp

Here, we are telling GROMACS to constrain the C-H bonds in our system (note that in the simulation of water the bonds and angles were constrained by default, but this default behavior only applies to water). The C-H bonds represent the fastest frequency motions in our system, and thus will be the limiting factor in determining how large we can make our timestep. In the absence of multi-timestep integration methods, another option for facilitating a larger timestep is to apply constraints to these bonds to remove the high frequency motions.

**NOTE:** Typically for all-atom simulations, the maximum timestep that can be used is 1fs. However, if hydrogen bonds are constrained, the maximum timestep can usually be increased to 2fs, thus doubling performance.

Okay, now let's run `grompp` on our system.

In [ ]:
!gmx grompp -v -f npt-alkane.mdp -c hexane.gro -p hexane.top -o npt-hexane

If `grompp` was successful, we can now perform the simulation with `mdrun`. Note, this system is a bit larger than the water box, so it will take a bit longer to complete. However, it should still complete within a few minutes.

In [ ]:
!gmx mdrun -s npt-hexane.tpr -o -x -deffnm npt-hexane

Again, let's extract quantities from the energy file and plot by executing the following two cells.

In [ ]:
!echo 11 12 13 18 19\\n0 | gmx energy -f npt-hexane.edr -o npt-hexane.xvg

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('npt-hexane.xvg', skiprows=27)

# Strip out the very beginning of the run
data = data[10:]

fig, ax = plt.subplots(5, 1)

properties = ['Total energy, kJ/mol', 'Temperature, K', 'Pressure, atm', 'Volume, nm^3', 'Density, kg/m^3']

for i, sub_ax in enumerate(ax):
    sub_ax.plot(data[:, 0], data[:, i + 1])
    sub_ax.set_title(properties[i])
    sub_ax.set_ylabel(properties[i])
    
fig.subplots_adjust(hspace=1.0)
fig.set_size_inches(4, 12)
plt.show()

Examine the plots above. Has our system reached equilibrium? Does our density match up with what we would expect (659 kg/m^3)?